# 📊 Which ML algorithm works best to predict customer churn? :Exploratory Analysis with BigQuery & Python 🚀

## **Introduction**

In this notebook, we will analyze customer churn data using Google BigQuery and Python. The dataset contains information about customers. the varibles in the dataset are:

1. customer_id
2. credit_score 
3. country
4. gender
5. age
6. tenure
7. balance
8. products_number
9. credit_card
10. active_member
11. estimated_salary
12. churn, used as the target. 1 if the client has left the bank during some period or 0 if he/she has not.

In this notebook we will set up the connection between kaggle and BigQuery, retrieve some insightful data trough queries in BigQuery, and perform some basic data analysis. We will also visualize the data using matplotlib and seaborn.


Now the steps to migrate the data from kaggle to bigquery are shown in this diagram: 

## Fetching Dataset From Kaggle

Now, this dataset is avaliable at kaggle [here](https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset). I selected this dataset because the dependent variable is categorical. As a chemist, I'm used to working with continuous variables for prediction problems. I find this dataset a good opportunity to apply  classification algorithms.

let's then import the `kaggle API` first:

In [3]:
from kaggle.api.kaggle_api_extended import KaggleApi


Now, the kaggle credentials are stored as secrets in this github repository, so we can extract them using `os.environ.get`. Then, we can create folder with the filepath where the json file will be stored.

In [4]:
import os

kaggle_token = os.environ.get("KAGGLE_JSON")
kaggle_config_dir = os.path.expanduser("~/.config/kaggle")
os.makedirs(kaggle_config_dir, exist_ok=True)



Once the file is ready to host our credentials with it's dedicated folder, we can create the write content into the file, with the filename.

the `with` statement is used to open the file in write mode, hence the use of the `w` flag. 

finally we set the permissions of the file so only me (the owner) can read and write to it, and then we can use the kaggle API to download the dataset.

In [5]:
kaggle_json_path = os.path.join(kaggle_config_dir, "kaggle.json")
with open(kaggle_json_path, "w") as f:
    f.write(kaggle_token)

os.chmod(kaggle_json_path, 0o600)

Now that we have everything set up, the next step would be to authenticate and check whether or not the credentials work:

In [6]:
api = KaggleApi()
api.authenticate()

There are no errors raised, great! Now if we provide python both the link to the dataset and the folder where it will be saved, as follows:

In [7]:
dataset_name = "gauravtopre/bank-customer-churn-dataset"  # the categorical dependent variable dataset I described to you earlier
download_path = "/tmp"  # Temporary location, not inside this repo

next we can use these new variables `dataset_name` and ``:

* Kaggle's API to download the dataset
* glob to check whether the file is downloaded or not, and display it's path:

In [8]:
import glob
# Download dataset (ZIP file)
api.dataset_download_files(dataset_name, path=download_path, unzip=True)

# Find the downloaded CSV file
csv_files = glob.glob(f"{download_path}/*.csv")
assert len(csv_files) > 0, "No CSV files found. Check dataset name."
csv_file_path = csv_files[0]  

print(f"✅ Dataset downloaded: {csv_file_path}")

Dataset URL: https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset
✅ Dataset downloaded: /tmp/Bank Customer Churn Prediction.csv
✅ Dataset downloaded: /tmp/Bank Customer Churn Prediction.csv


done! now we can move on to the next part, which is uploading the dataset to bigquery to excecute queries and explore the dataset. for this, we will use a gcp service account which I already set up, and saved it's key as a secret in this repository.

## Setting up Service Account Credentials and Initializing BigQuery Client

Since both the credentials for kaggle and the service account are stored as secrets in this repository, we can extract them using `os.environ.get`. 
Then, we can create folder with the filepath where the json file will be stored.

In [9]:
import os
# Save GCP service account JSON to a temporary file and set the path
gcp_key = os.environ.get("GCP_SA_KEY")
gcp_path = "/tmp/sa_credentials.json"
 
with open(gcp_path, "w") as f:
    f.write(gcp_key)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = gcp_path




The folling code initializes the BigQuery client using the service account credentials stored in the environment variable `GOOGLE_APPLICATION_CREDENTIALS`. This allows us to interact with BigQuery and run queries.


In [10]:
import google.cloud.bigquery as bigquery
client = bigquery.Client()
print(client.project)

kagglebigquerybankchurn


This `client` object will allow us to perform operations like:

- Running SQL queries on BigQuery Datasets (My main goal on pursuing this specific path)
- Creating datasets and tables (this will be needed to fetch)
- fetch query results as `pandas` DataFrames.

In [11]:
project_id = client.project # This is already defined by the service account
dataset_id = f"{project_id}.churn_analysis"
table_id = "Kaggle_churn"
full_table_id = f"{dataset_id}.{table_id}"
print(full_table_id)

kagglebigquerybankchurn.churn_analysis.Kaggle_churn


Now that we have defined that unique identifier for the data table, and the dataset whithin it will be saved, we can go ahead and create the dataset:

In [12]:
client.create_dataset(dataset_id, exists_ok=True)

Dataset(DatasetReference('kagglebigquerybankchurn', 'churn_analysis'))


Now check if there is any tables within the dataset. I think there should not because we have not used the kaggle API yet:

In [13]:
tables = {table.table_id for table in client.list_tables(dataset_id)}
print(tables)

{'Kaggle_churn'}


Ok this set comprehension (which I find tremendously efficient) loops trough all the tables within the dataset named `dataset_id`. indeed since right now we haven't used the kaggle functions there is no table yet. Let's get to that.

ok the next chunk of code will upload the dataset to bigquery. 

this process need several configurations to be set, so we are going to save these in the `job_config` variable as follows:

In [14]:
job_config = bigquery.LoadJobConfig(
    autodetect = True,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # This ensures table is overwritten, not appended to
)

### Important: Preventing Duplicate Data Accumulation

The `write_disposition` parameter is crucial for data integrity:

- **`WRITE_TRUNCATE`** (recommended): Overwrites the existing table with new data, preventing duplicates
- **`WRITE_APPEND`** (default): Adds new data to existing table, which can create duplicates on repeated runs
- **`WRITE_EMPTY`**: Only writes if the table is empty, fails if table exists

Since we want to ensure clean, non-duplicated data in our analysis, we use `WRITE_TRUNCATE` to replace the table contents on each upload. This is especially important when running notebooks multiple times during development.

that `job_config` variable will tell the `load_table_from_file` function which schema to use, and what configuration the table load should have.

I've been trough this process many times inside the bigquery site, and every time I felt like selecting options from drop down menus, and pushing buttons was not as reproducible as I would like it to be. Now I'm glad it can be written down in code and anyone can use this for their needs (assuming someone besides me reads this 🤣). 

Now let's do the upload:

In [15]:
with open(csv_file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, full_table_id, job_config=job_config)

job.result()

LoadJob<project=kagglebigquerybankchurn, location=US, id=763c724b-fb29-4b95-ad4b-623c41cca20d>

## SQL Queries in BigQuery

OK! finally, we have the dataset in biquery and we can query it so it gives us insight on how we should address the data, and which model we could use to predict the bank churn. 





## EDA with SQL Queries

### Data Validation

First we can check if there is any missing values. This can be done both with SQL and python, let's do both for sake of comparison.

#### Missing values with SQL

Since using SQL in bigquery there is no way to dinamically loop trough a dataset column, each column must be checked. if we were only using SQL it would look like this:

`SELECT 
    SUM(CASE WHEN credit_score IS NULL THEN 1 ELSE 0 END) AS missing_credit_score,
    ... -- other columns as needed`

Where the CASE statement begins a control flow structure that checks whether or not there is null values in the column, and then the sum fuction adds them all up. Having checked that, we can define the query to pass it to the BigQuery client and execute it:

In [16]:
missing_values_query = """
SELECT
  SUM(CASE WHEN credit_score IS NULL THEN 1 ELSE 0 END) AS missing_credit_score,
  SUM(CASE WHEN country IS NULL THEN 1 ELSE 0 END) AS missing_country,
  SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS missing_gender,
  SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS missing_age,
  SUM(CASE WHEN tenure IS NULL THEN 1 ELSE 0 END) AS missing_tenure,
  SUM(CASE WHEN balance IS NULL THEN 1 ELSE 0 END) AS missing_balance,
  SUM(CASE WHEN products_number IS NULL THEN 1 ELSE 0 END) AS missing_products_number,
  SUM(CASE WHEN credit_card IS NULL THEN 1 ELSE 0 END) AS missing_credit_card,
  SUM(CASE WHEN active_member IS NULL THEN 1 ELSE 0 END) AS missing_active_member,
  SUM(CASE WHEN estimated_salary IS NULL THEN 1 ELSE 0 END) AS missing_estimated_salary,
  SUM(CASE WHEN churn IS NULL THEN 1 ELSE 0 END) AS missing_churn
FROM
  `kagglebigquerybankchurn.churn_analysis.Kaggle_churn`
"""

### Generating data cleaning SQL queries with python:

above we just repeated the same structure that is neede to retrieve the amount of missing values with SQL in a column, for all columns within the dataset. Conversely, we can use python to automate query generation, as follows:

* First we use the bigquery `client` we generated to retrieve the table's metadata, where column names are stored:

In [17]:
table_ref = client.get_table(full_table_id)

* then, we can use a list comprehension, which is a compact loop that generates a list of strings, to save the column names in the columns variable:

In [18]:
columns = [schema_field.name for schema_field in table_ref.schema if schema_field.name != "customer_id"]
print(columns)

['credit_score', 'country', 'gender', 'age', 'tenure', 'balance', 'products_number', 'credit_card', 'active_member', 'estimated_salary', 'churn']


Now that we have the actual names of the columns, the same query can be generated using a list comprehension as follows:

In [19]:
select_expression = [f"SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS missing_{col}" for col in columns]
print(select_expression)

['SUM(CASE WHEN credit_score IS NULL THEN 1 ELSE 0 END) AS missing_credit_score', 'SUM(CASE WHEN country IS NULL THEN 1 ELSE 0 END) AS missing_country', 'SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS missing_gender', 'SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS missing_age', 'SUM(CASE WHEN tenure IS NULL THEN 1 ELSE 0 END) AS missing_tenure', 'SUM(CASE WHEN balance IS NULL THEN 1 ELSE 0 END) AS missing_balance', 'SUM(CASE WHEN products_number IS NULL THEN 1 ELSE 0 END) AS missing_products_number', 'SUM(CASE WHEN credit_card IS NULL THEN 1 ELSE 0 END) AS missing_credit_card', 'SUM(CASE WHEN active_member IS NULL THEN 1 ELSE 0 END) AS missing_active_member', 'SUM(CASE WHEN estimated_salary IS NULL THEN 1 ELSE 0 END) AS missing_estimated_salary', 'SUM(CASE WHEN churn IS NULL THEN 1 ELSE 0 END) AS missing_churn']


now that we have each of the lines for the query, we can join them together as follows:

In [20]:
missing_values_query_python =  f"""
SELECT
    {', '.join(select_expression)}
FROM
    `{full_table_id}`
"""
print(missing_values_query_python)


SELECT
    SUM(CASE WHEN credit_score IS NULL THEN 1 ELSE 0 END) AS missing_credit_score, SUM(CASE WHEN country IS NULL THEN 1 ELSE 0 END) AS missing_country, SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS missing_gender, SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS missing_age, SUM(CASE WHEN tenure IS NULL THEN 1 ELSE 0 END) AS missing_tenure, SUM(CASE WHEN balance IS NULL THEN 1 ELSE 0 END) AS missing_balance, SUM(CASE WHEN products_number IS NULL THEN 1 ELSE 0 END) AS missing_products_number, SUM(CASE WHEN credit_card IS NULL THEN 1 ELSE 0 END) AS missing_credit_card, SUM(CASE WHEN active_member IS NULL THEN 1 ELSE 0 END) AS missing_active_member, SUM(CASE WHEN estimated_salary IS NULL THEN 1 ELSE 0 END) AS missing_estimated_salary, SUM(CASE WHEN churn IS NULL THEN 1 ELSE 0 END) AS missing_churn
FROM
    `kagglebigquerybankchurn.churn_analysis.Kaggle_churn`



Now let's run both queries, `missing_values_query` and `missing_values_query_python`, to see if they yield the same results:

* First the query we wrote manually: 

In [21]:
hardcoded_job = client.query(missing_values_query)
hardcoded_result = hardcoded_job.to_dataframe()
display(hardcoded_result)

,missing_credit_score,missing_country,missing_gender,missing_age,missing_tenure,missing_balance,missing_products_number,missing_credit_card,missing_active_member,missing_estimated_salary,missing_churn
0,0,0,0,0,0,0,0,0,0,0,0


* and now the query generated with python:

In [22]:
python_job = client.query(missing_values_query_python)
python_result = python_job.to_dataframe()
display(python_result) 

,missing_credit_score,missing_country,missing_gender,missing_age,missing_tenure,missing_balance,missing_products_number,missing_credit_card,missing_active_member,missing_estimated_salary,missing_churn
0,0,0,0,0,0,0,0,0,0,0,0


Great! we can visually confirm that tere is no missing values in this dataset. if this process were to be automated, this visual inspection would not be enough. thankfully python allows us to compare the results of both queries, and check if they are equal:

In [23]:
comparison = hardcoded_result.equals(python_result)
print(f"do both queries give the same result:? {comparison}")

do both queries give the same result:? True


that's great, missing values is one of the first data integrity checks that should be done and now we have done it.

## Checking for duplicates in SQL

SQL makes it easy to check for duplicates as follows:

In [24]:
duplicates_query = """

SELECT customer_id, COUNT(*) AS count
FROM `kagglebigquerybankchurn.churn_analysis.Kaggle_churn`
GROUP BY customer_id
HAVING count > 1; 
"""

duplicates_job = client.query(duplicates_query)
duplicates_result = duplicates_job.to_dataframe()
print(f"Number of duplicate rows: {len(duplicates_result)}")


Number of duplicate rows: 0


### Checking for Complete Duplicate Rows (All Fields)

The previous query only checked for duplicate customer IDs. But we need to verify if rows with duplicate customer IDs also have identical content across ALL fields. Let's check this:

In [25]:
# SQL query to check for complete duplicate rows (all fields identical)
complete_duplicates_query = """
SELECT 
    customer_id, credit_score, country, gender, age, tenure, balance, 
    products_number, credit_card, active_member, estimated_salary, churn,
    COUNT(*) as duplicate_count
FROM `kagglebigquerybankchurn.churn_analysis.Kaggle_churn`
GROUP BY 
    customer_id, credit_score, country, gender, age, tenure, balance, 
    products_number, credit_card, active_member, estimated_salary, churn
HAVING COUNT(*) > 1
ORDER BY duplicate_count DESC
"""

complete_duplicates_job = client.query(complete_duplicates_query)
complete_duplicates_result = complete_duplicates_job.to_dataframe()

print(f"Number of unique row combinations that appear more than once: {len(complete_duplicates_result)}")
print(f"Total duplicate rows (sum of all duplicates): {complete_duplicates_result['duplicate_count'].sum()}")

# Show first few examples
if len(complete_duplicates_result) > 0:
    print("\nFirst 5 complete duplicate row combinations:")
    display(complete_duplicates_result.head())

Number of unique row combinations that appear more than once: 0
Total duplicate rows (sum of all duplicates): 0


the above code identified that there is 10000 customer ids that appear more than once.

In [26]:
# Let's compare the results
print("=== COMPARISON ===")
print(f"Customer IDs with duplicates: {len(duplicates_result)}")
print(f"Complete duplicate row combinations: {len(complete_duplicates_result)}")

# Check if they match
if len(duplicates_result) == len(complete_duplicates_result):
    print("✅ All duplicate customer IDs have identical content across all fields!")
    print("✅ Safe to remove duplicates - they are true duplicates")
else:
    print("⚠️  Some customer IDs have different content - need to investigate further")
    print("⚠️  Cannot safely remove duplicates without further analysis")

=== COMPARISON ===
Customer IDs with duplicates: 0
Complete duplicate row combinations: 0
✅ All duplicate customer IDs have identical content across all fields!
✅ Safe to remove duplicates - they are true duplicates


now we can retrieve the complete dataset using the following query:



In [27]:
df_query   = """
SELECT *
FROM `kagglebigquerybankchurn.churn_analysis.Kaggle_churn`
"""
df_job = client.query(df_query)
df = df_job.to_dataframe()

### Python Duplicate Analysis

Now let's perform the same duplicate analysis using Python with pandas to compare the power and ease of both approaches:

In [28]:
# Python duplicate analysis using the full dataset (df)
print("=== PYTHON DUPLICATE ANALYSIS ===")

# 1. Check for duplicate customer IDs
customer_id_duplicates = df.groupby('customer_id').size()
customer_ids_with_duplicates = customer_id_duplicates[customer_id_duplicates > 1]

print(f"Customer IDs with duplicates: {len(customer_ids_with_duplicates)}")

# 2. Check for complete duplicate rows (all fields identical)
complete_duplicates = df.duplicated(keep=False)  # keep=False marks all duplicates as True
duplicate_rows = df[complete_duplicates]

print(f"Total rows that are complete duplicates: {complete_duplicates.sum()}")
print(f"Unique combinations of complete duplicates: {len(df[df.duplicated(keep='first')])}")

# 3. Compare customer ID duplicates vs complete row duplicates
print("\n=== COMPARISON WITH SQL RESULTS ===")
print(f"SQL - Customer IDs with duplicates: {len(duplicates_result)}")
print(f"Python - Customer IDs with duplicates: {len(customer_ids_with_duplicates)}")
print(f"Match: {'✅' if len(duplicates_result) == len(customer_ids_with_duplicates) else '❌'}")

# 4. Check if all customer ID duplicates are also complete row duplicates
if complete_duplicates.sum() > 0:
    # Group complete duplicates by customer_id to see the pattern
    duplicate_customer_analysis = duplicate_rows.groupby('customer_id').size()
    print(f"\nComplete duplicate rows grouped by customer_id:")
    print(f"Number of customer IDs in complete duplicates: {len(duplicate_customer_analysis)}")
    
    # Verify if duplicate customer IDs have identical content
    if len(customer_ids_with_duplicates) == len(duplicate_customer_analysis):
        print("✅ All duplicate customer IDs have identical content across all fields!")
        print("✅ Safe to remove duplicates using pandas.drop_duplicates()")
    else:
        print("⚠️  Some customer IDs have different content - investigate further!")
else:
    print("No complete duplicate rows found.")

=== PYTHON DUPLICATE ANALYSIS ===
Customer IDs with duplicates: 0
Total rows that are complete duplicates: 0
Unique combinations of complete duplicates: 0

=== COMPARISON WITH SQL RESULTS ===
SQL - Customer IDs with duplicates: 0
Python - Customer IDs with duplicates: 0
Match: ✅
No complete duplicate rows found.


### Removing Duplicates Safely

If our analysis confirms that duplicate customer IDs have identical content across all fields, we can safely remove duplicates. Here's how to do it in both SQL and Python:

In [29]:
# Method 1: SQL - Create a deduplicated dataset using ROW_NUMBER()
dedup_sql_query = """
SELECT 
    customer_id, credit_score, country, gender, age, tenure, balance, 
    products_number, credit_card, active_member, estimated_salary, churn
FROM (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY customer_id
            ORDER BY customer_id
        ) as row_num
    FROM `kagglebigquerybankchurn.churn_analysis.Kaggle_churn`
) 
WHERE row_num = 1
"""

# Execute the query to get deduplicated data
dedup_sql_job = client.query(dedup_sql_query)
dedup_sql_df = dedup_sql_job.to_dataframe()

print(f"Original dataset size: {len(df)}")
print(f"Deduplicated dataset size (SQL): {len(dedup_sql_df)}")
print(f"Rows removed: {len(df) - len(dedup_sql_df)}")

Original dataset size: 10000
Deduplicated dataset size (SQL): 10000
Rows removed: 0


In [29]:
# Method 2: Python - Remove duplicates using pandas
dedup_python_df = df.drop_duplicates()

print(f"\nPython deduplication:")
print(f"Original dataset size: {len(df)}")
print(f"Deduplicated dataset size (Python): {len(dedup_python_df)}")
print(f"Rows removed: {len(df) - len(dedup_python_df)}")

# Verify both methods give the same result
print(f"\nBoth methods give same result: {'✅' if len(dedup_sql_df) == len(dedup_python_df) else '❌'}")

# Final verification - check unique customer IDs in deduplicated data
print(f"\nFinal verification:")
print(f"Unique customer IDs in SQL deduplicated data: {dedup_sql_df['customer_id'].nunique()}")
print(f"Unique customer IDs in Python deduplicated data: {dedup_python_df['customer_id'].nunique()}")
print(f"Total rows in deduplicated data: {len(dedup_python_df)}")

# If they match, we have successfully removed duplicates
if dedup_python_df['customer_id'].nunique() == len(dedup_python_df):
    print("✅ Perfect! Each customer ID now appears exactly once.")
else:
    print("⚠️  Still some issues with the deduplication process.")


Python deduplication:
Original dataset size: 10000
Deduplicated dataset size (Python): 10000
Rows removed: 0

Both methods give same result: ✅

Final verification:
Unique customer IDs in SQL deduplicated data: 10000
Unique customer IDs in Python deduplicated data: 10000
Total rows in deduplicated data: 10000
✅ Perfect! Each customer ID now appears exactly once.


### Summary: SQL vs Python for Duplicate Handling

#### **SQL Approach:**
**Advantages:**
- ✅ Handles large datasets efficiently (server-side processing)
- ✅ Explicit control over duplicate detection logic
- ✅ Can be integrated into ETL pipelines
- ✅ Memory efficient for very large datasets

**Disadvantages:**
- ❌ More verbose syntax (need to specify all columns)
- ❌ Less flexible for complex duplicate detection rules
- ❌ Requires good understanding of window functions

#### **Python/Pandas Approach:**
**Advantages:**
- ✅ Simple, concise syntax (`df.drop_duplicates()`)
- ✅ Flexible - easy to specify subset of columns or custom logic
- ✅ Integrated with data analysis workflow
- ✅ Rich ecosystem for further data validation

**Disadvantages:**
- ❌ Memory intensive for very large datasets
- ❌ Slower for massive datasets compared to SQL
- ❌ Requires loading full dataset into memory

#### **Best Practice Recommendation:**
- Use **SQL** for initial data cleaning in production ETL pipelines
- Use **Python** for exploratory analysis and validation
- Always validate that both approaches give consistent results (as we did above)

there is a hundred thousand total rows, and just ten thousand unique customer ids, that could mean that each customer id has 10 entries. 

## Exploratory data analysis in python

now let's see how the variables are related to the churn rate. For that, we will create the main dataframe again, but will not limit the amount of rows to ten, this dataframe will be named `full_df`


In [30]:
full_query = """
SELECT *
FROM `kagglebigquerybankchurn.churn_analysis.Kaggle_churn`
"""
full_query_job = client.query(full_query)
full_df = full_query_job.to_dataframe()


let's really quick check if there are any missing values:

In [31]:
full_df.isna().sum()

customer_id         0
credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
dtype: int64

Now we can check information about the variables with the `.info` and `.describe` functions as follows:

In [32]:
full_df.describe()


,customer_id,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
count,10000.0,10000.0,10000.0,10000.0,10000.000000,10000.0,10000.0,10000.0,10000.000000,10000.0
mean,15690940.5694,650.5288,38.9218,5.0128,76485.889288,1.5302,0.7055,0.5151,100090.239881,0.2037
std,71936.186123,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,15565701.0,350.0,18.0,0.0,0.000000,1.0,0.0,0.0,11.580000,0.0
25%,15628528.25,584.0,32.0,3.0,0.000000,1.0,0.0,0.0,51002.110000,0.0
50%,15690738.0,652.0,37.0,5.0,97198.540000,1.0,1.0,1.0,100193.915000,0.0
75%,15753233.75,718.0,44.0,7.0,127644.240000,2.0,1.0,1.0,149388.247500,0.0
max,15815690.0,850.0,92.0,10.0,250898.090000,4.0,1.0,1.0,199992.480000,1.0


Thanks to this summary table, we can identify that there is two groups of variables:

### Continuous variables:
- Credit_score
- Age
- Estimated_salary
- Balance

Let's examine these variables further by visualizing their distributions:

In [33]:
# setting the canvas for four plots (2 rows, 4 columns)
import matplotlib.pyplot as plt
import seaborn as sns

# Set up a larger figure with a grid of 4x2 subplots
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(4, 2, height_ratios=[3, 1, 3, 1])

# Set the figure title
fig.suptitle('Continuous Variables Distributions', fontsize=20, y=0.95)

# Define variables and titles for the plots
variables = ['credit_score', 'age', 'estimated_salary', 'balance']
titles = ['Credit Score Distribution', 'Age Distribution', 'Estimated Salary Distribution', 'Balance Distribution']

# Loop through variables and plot
for i, (var, title) in enumerate(zip(variables, titles)):
    # Histogram
    ax_hist = fig.add_subplot(gs[i // 2 * 2, i % 2])
    sns.histplot(data=full_df, x=var, kde=True, ax=ax_hist)
    ax_hist.set_title(title)
    
    # Boxplot
    ax_box = fig.add_subplot(gs[i // 2 * 2 + 1, i % 2])
    sns.boxplot(data=full_df, x=var, color='orange', ax=ax_box)
    ax_box.set_title('')

# Adjust layout to prevent overlap
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Show the plots
plt.show()


ModuleNotFoundError: No module named 'matplotlib'

Thanks to these visualizations we can comment some things about these four continuous variables, to consider in the modelling stage:

1. **Credit Score**
    - Is normally distributed, good for modelling, and also complies with the assumptions of most statistical tests.
    - Supports the assumptions of most hypothesis tests (that's great, just What I Learned in the regression course of the *Google Advanced Data Analytics Professional Certificate*).

    **References**:
    - Hosmer Jr, D. W., et al. (2013). ["Applied Logistic Regression"](https://books.google.com.co/books?hl=en&lr=&id=bRoxQBIZRd4C&oi=fnd&pg=PR13&dq=Hosmer+Jr,+D.+W.,+et+al.+(2013).+%22Applied+Logistic+Regression%22+-+Discusses+normality+assumptions&ots=kM4Otl6Sb5&sig=9Q0GI1KojiWmqTd861azWkxeWm8&redir_esc=y#v=onepage&q&f=false) - Discusses normality assumptions.
    - Altman, N., & Krzywinski, M. (2016). ["Points of Significance: Analyzing outliers"](https://go.gale.com/ps/i.do?id=GALE%7CA461963379&sid=googleScholar&v=2.1&it=r&linkaccess=abs&issn=15487091&p=AONE&sw=w&userGroupName=anon%7E5d49140c&aty=open-web-entry) Nature Methods.

2. **Age (normal with a right tail)**
    - Age distributions are commonly found to have a right-skewed distribution (this makes sense to me because older people tend to die and then the amount of older people decreases).
    - May require log transformation for a linear model.

    **References**:
    - Cox, D. R., & Snell, E. J. (1989). ["Analysis of Binary Data"](https://www.taylorfrancis.com/books/mono/10.1201/9781315137391/analysis-binary-data-cox) (2nd ed.). Chapman and Hall/CRC.
    - Faraway, J. J. (2016). ["Extending the Linear Model with R"](https://www.google.com.co/books/edition/Extending_the_Linear_Model_with_R/XAzYCwAAQBAJ?hl=en&gbpv=1&dq=Faraway,+J.+J.+(2016).+%22Extending+the+Linear+Model+with+R%22+(2nd+ed.),+Chapman+and+Hall/CRC&pg=PR2&printsec=frontcover) (2nd ed.), Chapman and Hall/CRC.

3. **Estimated Salary (Uniform)**
    - Uniform distribution. This is weird for me, and checking in literature uniform distributions indeed are due to:
        - Data preprocessing/binning could work to enhance its usability.
        - Potential sampling bias.

    **References**:
    - Cohen, J., et al. (2003). "Applied Multiple Regression/Correlation Analysis."
    - Friedman, J., et al. (2001). "The Elements of Statistical Learning" - Discussion of feature distributions.

4. **Balance (Normal, with many values around zero)**
    - Well, it seems like in financial data this is normal, and I understand, many people will have accounts with zero balance.
    - Statistical implications:
        - Need for specialized modeling approaches.
        - Consider two-part models (zero vs non-zero).
        - May require transformation for non-zero values.

    **References**:

    - Min, Y., & Agresti, A. (2002). "Modeling nonnegative data with clumping at zero: A survey." Journal of the Iranian Statistical Society, 1(1-2), 7-33.


    - Fletcher, D., & Dixon, P. M. (2012). "Modelling data with excess zeros and measurement error: Application to evaluating relationships between abundances of multiple species." Biometrics, 68(1), 270-278.


    - Tu, W. (2006). "Zero-Inflated Data." Encyclopedia of Environmetrics, 6.




As a summary, what we can do with these variables is: 
- **Credit Score:** Use as-is in logistic regression
- **Age:** Consider log transformation
- **Salary:** Consider binning or categorical transformation
- **Balance:** Create binary flag for zero values

Before we move on to categorical variables, I want to see how the dependent variable, churn, is related to these four variables in a similar fashion, son now we will again draw four plots in the same canvas, but this time the will be scatter plots instead of histograms, keeping the boxplot of the independent variable at the bottom:

In [ ]:
# setting the canvas for four plots (2 rows, 4 columns)
import matplotlib.pyplot as plt
import seaborn as sns

# Set up a larger figure with a grid of 4x2 subplots
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(4, 2, height_ratios=[3, 1, 3, 1])

# Set the figure title
fig.suptitle('Churn Rate vs Continuous Variables', fontsize=20, y=0.95)

# Define variables and titles for the plots
variables = ['credit_score', 'age', 'estimated_salary', 'balance']
titles = ['Churn Rate vs Credit Score', 'Churn Rate vs Age', 'Churn Rate vs Estimated Salary', 'Churn Rate vs Balance']

# Loop through variables and plot
for i, (var, title) in enumerate(zip(variables, titles)):
    # Scatterplot
    ax_scatter = fig.add_subplot(gs[i // 2 * 2, i % 2])
    sns.scatterplot(data=full_df, x=var, y='churn', alpha=0.5, ax=ax_scatter)
    ax_scatter.set_title(title)
    ax_scatter.set_ylabel('Churn Rate')
    
    # Boxplot
    ax_box = fig.add_subplot(gs[i // 2 * 2 + 1, i % 2])
    sns.boxplot(data=full_df, x=var, color='orange', ax=ax_box)
    ax_box.set_title('')

# Adjust layout to prevent overlap
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Show the plots
plt.show()


## Categorical variables: 


In [1]:
# Set up a larger figure with a grid of 3x2 subplots
fig, axes = plt.subplots(3, 2, figsize=(20, 15))

# Set the figure title
fig.suptitle('Churn Rate vs Categorical Variables', fontsize=20, y=0.95)

# Define categorical variables and titles for the plots
cat_variables = ['gender', 'country', 'credit_card', 'active_member', 'products_number', 'tenure']
titles = ['Churn vs Gender', 'Churn vs Country', 'Churn vs Credit Card', 
          'Churn vs Active Member', 'Churn vs Products Number', 'Churn vs Tenure']

# Loop through variables and plot
for ax, var, title in zip(axes.flatten(), cat_variables, titles):
    sns.barplot(data=full_df, x=var, y='churn', ax=ax)
    ax.set_title(title)
    ax.set_ylabel('Churn Rate')
    ax.set_xlabel(var.capitalize())

# Adjust layout to prevent overlap
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Show the plots
plt.show()


NameError: name 'plt' is not defined

## Analysis of Categorical Variables and Their Relationship with Churn

### Key Observations and Modeling Implications:

1. **Gender**
   - Slight imbalance in distribution (more females than males)
   - Similar churn rates between groups
   - **Modeling Decision**: Include as-is, but consider:
     - Using stratification in train-test split
     - Checking for interaction effects with other variables

2. **Country**
   - Uneven distribution (Germany ~2x France/Spain)
   - Different churn patterns across countries
   - **Modeling Decision**: 
     - Convert to dummy variables
     - Consider weighted sampling if using tree-based models
     - Important feature to keep due to visible variation in churn rates

3. **Credit Card Status**
   - Balanced distribution
   - Similar churn rates between holders/non-holders
   - **Modeling Decision**: Consider excluding due to low predictive power
     - Run feature importance check to confirm
     - Could be kept for interaction effects

4. **Active Member Status**
   - Strong predictor: 2x churn rate for inactive members
   - Clear separation between groups
   - **Modeling Decision**: 
     - Definitely keep
     - Consider as primary feature
     - Check for interactions with balance and products_number

5. **Products Number**
   - Higher churn for 3-4 products
   - Non-linear relationship with churn
   - **Modeling Decision**:
     - Keep as categorical rather than continuous
     - Consider binning into: low (1-2) and high (3-4) products
     - Test both original and binned versions

6. **Tenure**
   - Uniform distribution across groups
   - Weak relationship with churn
   - **Modeling Decision**:
     - Test both as continuous and categorical
     - Consider binning into meaningful groups (new/established customers)
     - Check for interaction with active_member
     




## Next Steps: Modular Functions for Data Upload and Validation

To make our workflow more robust and reusable, let's work together to build modular functions for:
- Uploading data to BigQuery safely
- Validating data quality (missing values, duplicates, etc.)

We'll develop these step by step in the next cells, so you can learn and participate in the process.